In [2]:
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
import numpy as np
import pandas as pd
import re
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [8]:
oft_in_file = "../../../data/CS/feature_encodings/baselines/OFT/objects.csv"

In [9]:
objects_df = pd.read_csv(oft_in_file)

In [23]:
objects_df =objects_df[objects_df["ea4"] != 0]
objects_df["ea4"] = objects_df["ea4"] - 1
objects_df

,ocel:oid,oa1,oa2,oa3,oa4,oa5,oa6,oa7,oa8,oa9,oa10,oa11,oa12,oa13,oa14,oa15,oa16,ea4
2,KRV-1872351,0.368881,1.378472,0.689236,0.459491,-0.481001,0.459491,0.459491,0.459491,0.459491,1.378472,0.689236,1.378472,1.378472,NaN,1.378472,0.459491,2.0
6,KRV-3631161,0.368881,0.875694,0.875694,0.625496,-0.741550,0.625496,0.625496,0.625496,1.378472,2.189236,1.459491,0.229745,0.875694,NaN,0.344618,1.378472,3.0
13,KRV-9293901,0.368881,0.698206,0.930941,0.598462,-0.481001,1.378472,0.598462,0.598462,0.336806,2.792824,2.094618,0.413729,0.698206,NaN,0.672309,0.336806,3.0
16,KRV-2308561,0.368881,0.825231,1.125316,0.728145,0.691471,0.523654,0.728145,0.728145,2.689236,0.125316,0.106036,0.586154,0.825231,NaN,0.196925,2.689236,5.0
27,KRV-2656400,0.368881,0.765770,1.148654,0.680684,0.691471,0.532536,0.656374,0.656374,3.792824,0.351356,0.307436,0.591480,0.765770,NaN,0.527034,3.792824,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420728,KRS-2447124,0.368881,1.972525,1.181532,1.793449,0.821745,1.308467,1.267965,1.372407,1.422956,1.119607,1.131874,0.950592,1.545852,NaN,1.124205,1.423560,2.0
420729,KRS-6830105,0.368881,1.972530,1.181539,1.793453,-0.741550,1.308471,1.267969,1.372411,1.422962,1.634670,1.769004,1.394517,1.545856,NaN,1.124211,1.423566,3.0
420730,KRS-4000917,-1.129970,1.972538,1.181550,1.793461,-1.002100,2.785327,1.267976,1.372418,1.291373,1.634693,1.769017,0.950611,1.545863,NaN,1.124219,1.252017,2.0
420767,KRS-4024716,-1.129970,0.885146,1.181392,1.793278,-0.611276,1.308360,1.267870,1.372304,1.422864,1.634499,1.768776,1.560797,0.497770,NaN,1.124108,1.423468,3.0


In [24]:
# make train test split
X, y = (
    objects_df.drop(columns=["ea4", "ocel:oid"]),
    objects_df.loc[:, "ea4"],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=0
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [25]:
objects_df['ea4'].value_counts()

5.0    42023
3.0    35565
4.0    30988
2.0     6133
0.0     5069
1.0     3684
Name: ea4, dtype: int64

In [26]:
params = {
    "objective": "multiclass",
    "num_class": 6,
    "metric": ["multi_logloss"],
}
bst = lgb.train(
    params,
    train_data,
    # num_boost_round=5000,
    valid_sets=[valid_data],
    # callbacks=[lgb.early_stopping(50)],
)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.080640 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3355
[LightGBM] [Info] Number of data points in the train set: 98769, number of used features: 16
[LightGBM] [Info] Start training from score -3.183017
[LightGBM] [Info] Start training from score -3.522228
[LightGBM] [Info] Start training from score -3.001306
[LightGBM] [Info] Start training from score -1.245304
[LightGBM] [Info] Start training from score -1.382222
[LightGBM] [Info] Start training from score -1.077603
[1]	valid_0's multi_logloss: 1.3841
[2]	valid_0's multi_logloss: 1.34474
[3]	valid_0's multi_logloss: 1.31379
[4]	valid_0's multi_logloss: 1.28906
[5]	valid_0's multi_logloss: 1.26814
[6]	valid_0's multi_logloss: 1.25
[7]	valid_0's multi_logloss: 1.23484
[8]	valid_0's multi_logloss: 1.222
[9]	valid_0's multi_logloss: 1.21028
[10]	valid_0's multi_logloss: 1.19978
[11]	valid_0's mult

In [27]:
y_train_probs = bst.predict(X_train)
y_valid_probs = bst.predict(X_valid)

In [28]:
def get_predictions(lst):
    return max(range(len(lst)), key=lst.__getitem__)


y_train_preds = np.apply_along_axis(get_predictions, axis=1, arr=y_train_probs)
y_valid_preds = np.apply_along_axis(get_predictions, axis=1, arr=y_valid_probs)

In [29]:
train_mse_loss = metrics.accuracy_score(y_train, y_train_preds)
valid_mse_loss = metrics.accuracy_score(y_valid, y_valid_preds)

print(f"Training accuracy: {train_mse_loss}")
print(f"Validation accuracy: {valid_mse_loss}")

# including class '0': 79% accuracy (with highest prior of 0.7 for class '0')
# excluding class '0': 57% accuracy (with highest prior of 0.34 for class '5', which is actually 6)

Training accuracy: 0.6099282163431846
Validation accuracy: 0.5785040294820395


In [30]:
metrics.confusion_matrix(y_valid,y_valid_preds)

array([[ 418,   15,    0,  292,   98,  151],
       [  67,   92,   13,  323,  135,  137],
       [  34,   24,  221,  513,  177,  253],
       [  25,   16,   32, 4786, 1051, 1224],
       [  18,    9,   12, 1606, 2781, 1769],
       [  19,   10,    9, 1399,  977, 5987]])

In [31]:
objects_df['ea4'].value_counts(normalize=True).sort_index()

0.0    0.041057
1.0    0.029839
2.0    0.049675
3.0    0.288064
4.0    0.250992
5.0    0.340372
Name: ea4, dtype: float64